# Imports

In [78]:
import pandas as pd
import json

%matplotlib inline

# Load the data

In [79]:
with open('articles.json',encoding='utf8') as f:
    json_data = json.load(f)
articles_list = json_data['data'].values()
df = pd.DataFrame(articles_list)

In [0]:
df.head()

In [80]:
df.info()

,title,long_title,short_description,category,main_reporter,all_reporters,num_of_reporters,publish_timedate,last_update_timedate,content,views,article_interest,url
0,אחרי הפיגועים בבירה - התכנית של הממשלה,עמודי בטון יוצבו בתחנות האוטובוס: אחרי הפיגועי...,"בהודעה משותפת של ראש הממשלה ועיריית ירושלים, ש...",ביטחוני,ניר דבורי,[ניר דבורי],1,14/02/23 16:20,,"מיו""ש - לירושלים: ראש הממשלה בנימין נתניהו ועי...",13460,"{'interested': '315', 'not_interested': '31'}",https://www.mako.co.il/news-military/2023_q1/A...
1,אסיל סואעד שנהרג בפיגוע בשועפאט הובא למנוחות,"""אדם שמח, שתמיד נכון לסייע"": לוחם מג""ב אסיל סו...",בבית העלמין בכפר חוסנייה שבגליל הובא למנוחות ס...,ביטחוני,אדר גיציס,[אדר גיציס],1,14/02/23 15:55,,"סמ""ר אסיל סואעד, לוחם מג""ב שנהרג אמש מירי במהל...",6254,"{'interested': '125', 'not_interested': '2'}",https://www.mako.co.il/news-military/2023_q1/A...
2,"בקשות הנשק מכפילות את עצמן, אך פחות מאושרות",מרכז המחקר בכנסת חושף: מספר בקשות רישיון הנשק ...,בשנתיים האחרונות מספר האנשים שמגיש בקשה לרישיו...,ביטחוני,משה נוסבאום,[משה נוסבאום],1,14/02/23 13:57,,מספר הבקשות לרישיון נשק מכפיל את עצמו מידי שנה...,2649,"{'interested': '53', 'not_interested': '1'}",https://www.mako.co.il/news-military/2023_q1/A...
3,ישראלי שנכנס עם רכבו בטעות לקלנדיה והותקף,"""ניסיתי להגיע לכותל המערבי ועשו עליי לינץ'"": י...",עומר נחמן בן ה-35 נפצע באורח קל כתוצאה מהתקיפה...,ביטחוני,ענבר טויזר,"[ענבר טויזר, משה נוסבאום]",2,14/02/23 11:07,14/02/23 12:03,עומר נחמן בן ה-35 הותקף היום (שלישי) לאחר שככל...,9886,"{'interested': '254', 'not_interested': '6'}",https://www.mako.co.il/news-military/2023_q1/A...
4,מה מוביל את המחבלים הילדים והמוקש במזרח י-ם,"מה מוביל את ""הילדים מחבלים"" - והמוקש שעלול להד...",הפיגועים שבוצעו על ידי נערים צעירים מזכירים את...,ביטחוני,ניר דבורי,[ניר דבורי],1,14/02/23 09:36,,גילם הצעיר של המחבלים שביצעו את הפיגועים אתמול...,7879,"{'interested': '197', 'not_interested': '7'}",https://www.mako.co.il/news-military/2023_q1/A...


# Clean the data

In [81]:
df_cleaned = df.copy()

# add a column for the number_of_words of the articles
df_cleaned['number_of_words'] = df_cleaned['content'].apply(lambda x: len(x.split()))

# split needed columns
df_cleaned[['publish_date', 'publish_time']] = df_cleaned.publish_timedate.str.split(" ", expand = True)
df_cleaned[['interested', 'not_interested']] = df_cleaned['article_interest'].apply(pd.Series)

# converting string to DateTime
df_cleaned['publish_date'] = pd.to_datetime(df_cleaned['publish_date'])
df_cleaned['publish_date_day'] = df_cleaned.publish_date.dt.day_name()
df_cleaned['publish_time'] = pd.to_datetime(df_cleaned['publish_time'], format='%H:%M')

Changing the 'publish_time' column into 4 groups:
1- 00:00-06:00</li>
2- 06:00-12:00</li>
3- 12:00-18:00</li>
4- 18:00-00:00</li>

In [82]:
# define the bin edges for the groups
bin_edges = [pd.to_datetime('00:00', format='%H:%M'), pd.to_datetime('06:00', format='%H:%M'), pd.to_datetime('12:00', format='%H:%M'), pd.to_datetime('18:00', format='%H:%M'), pd.to_datetime('23:59', format='%H:%M')]

# create a new column with group numbers
df_cleaned['time_group'] = pd.cut(df_cleaned['publish_time'], bins=bin_edges, labels=[1, 2, 3, 4])

In [83]:
# replacing every odd or '0' value into None, so we can drop the rows that have None in it
df_cleaned['main_reporter'] = df_cleaned['main_reporter'].replace('N12', None)
df_cleaned['interested'] = df_cleaned['interested'].replace('0', None)
df_cleaned['not_interested'] = df_cleaned['not_interested'].replace('0', None)
df_cleaned['number_of_words'] = df_cleaned['number_of_words'].replace('0', None)

df_cleaned = df_cleaned.dropna(axis=0)

# define a list of categories to keep, and delete rows that are not included in the list
keep_categories = ['תרבות', 'כלכלה', 'פוליטי', 'ביטחוני', 'פלילי', 'בעולם']
index = df_cleaned['category'].apply(lambda x: any(word in x for word in keep_categories))
df_cleaned = df_cleaned[index]

# changing the weekdays column into numbers, so it will be easier to work with
weekday_to_number = {
    'Sunday': 1,
    'Monday': 2,
    'Tuesday': 3,
    'Wednesday': 4,
    'Thursday': 5,
    'Friday': 6,
    'Saturday': 7
}
df_cleaned['publish_date_day'] = df_cleaned['publish_date_day'].replace(weekday_to_number)

# deleting the rows that contains podcasts
keyword ='אחד ביום'
bool_index = df_cleaned['title'].str.contains(keyword)
df_cleaned = df_cleaned[~bool_index]


After splitting the relevant columns and getting the iformation we have needed we can delete the irelevant columns

In [84]:
df_cleaned = df_cleaned.drop('all_reporters', axis=1)
df_cleaned=df_cleaned.drop('last_update_timedate', axis=1)
df_cleaned=df_cleaned.drop('short_description', axis=1)
df_cleaned=df_cleaned.drop('publish_timedate', axis=1)
df_cleaned=df_cleaned.drop('publish_time', axis=1)
df_cleaned=df_cleaned.drop('publish_date', axis=1)
df_cleaned=df_cleaned.drop('article_interest', axis=1)


In [85]:
# convert relevant columns into int
df_cleaned['interested'] = df_cleaned['interested'].astype(int)
df_cleaned['not_interested'] = df_cleaned['not_interested'].astype(int)

In [0]:
df_cleaned.info()

In [87]:
df_cleaned.head()

,title,long_title,category,main_reporter,num_of_reporters,content,views,url,number_of_words,interested,not_interested,publish_date_day,time_group
0,אחרי הפיגועים בבירה - התכנית של הממשלה,עמודי בטון יוצבו בתחנות האוטובוס: אחרי הפיגועי...,ביטחוני,ניר דבורי,1,"מיו""ש - לירושלים: ראש הממשלה בנימין נתניהו ועי...",13460,https://www.mako.co.il/news-military/2023_q1/A...,403,315,31,3,3
1,אסיל סואעד שנהרג בפיגוע בשועפאט הובא למנוחות,"""אדם שמח, שתמיד נכון לסייע"": לוחם מג""ב אסיל סו...",ביטחוני,אדר גיציס,1,"סמ""ר אסיל סואעד, לוחם מג""ב שנהרג אמש מירי במהל...",6254,https://www.mako.co.il/news-military/2023_q1/A...,379,125,2,3,3
2,"בקשות הנשק מכפילות את עצמן, אך פחות מאושרות",מרכז המחקר בכנסת חושף: מספר בקשות רישיון הנשק ...,ביטחוני,משה נוסבאום,1,מספר הבקשות לרישיון נשק מכפיל את עצמו מידי שנה...,2649,https://www.mako.co.il/news-military/2023_q1/A...,162,53,1,3,3
3,ישראלי שנכנס עם רכבו בטעות לקלנדיה והותקף,"""ניסיתי להגיע לכותל המערבי ועשו עליי לינץ'"": י...",ביטחוני,ענבר טויזר,2,עומר נחמן בן ה-35 הותקף היום (שלישי) לאחר שככל...,9886,https://www.mako.co.il/news-military/2023_q1/A...,164,254,6,3,2
4,מה מוביל את המחבלים הילדים והמוקש במזרח י-ם,"מה מוביל את ""הילדים מחבלים"" - והמוקש שעלול להד...",ביטחוני,ניר דבורי,1,גילם הצעיר של המחבלים שביצעו את הפיגועים אתמול...,7879,https://www.mako.co.il/news-military/2023_q1/A...,264,197,7,3,2


In [86]:
df_cleaned.to_csv("C:/Users/golds/Documents/Studies/hit/Year B/semester A/Introduction to Data Science/Final Project/df_cleaned1.csv", index = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3174 entries, 0 to 4214
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   title             3174 non-null   object  
 1   long_title        3174 non-null   object  
 2   category          3174 non-null   object  
 3   main_reporter     3174 non-null   object  
 4   num_of_reporters  3174 non-null   int64   
 5   content           3174 non-null   object  
 6   views             3174 non-null   int64   
 7   url               3174 non-null   object  
 8   number_of_words   3174 non-null   int64   
 9   interested        3174 non-null   int32   
 10  not_interested    3174 non-null   int32   
 11  publish_date_day  3174 non-null   int64   
 12  time_group        3174 non-null   category
dtypes: category(1), int32(2), int64(4), object(6)
memory usage: 300.9+ KB
